# Tries
## [Palindrome pairs](https://leetcode.com/problems/palindrome-pairs/)
Given a list of strings `s`, find all the distince indices `i,j` such that the concatenation of `s[i]` and `s[j]` is a palindrome.
### Approach
* Brute force intuition:
    * For each string, try pairing it with every other string & check if it's a palindrome
    * $O(n2^n)$ runtime...
* What could make it better? 
    * For two strings to form a palindrome, their start & end letters have to be equal 
    * For each candidate string, we want to prune the potential partner strings we look at based on each letter
    * Ex: If my initial string is "abab" and I go through the letters sequentially:
        * At letter "a" - I can ignore all partners that don't end with "a"
        * At "ab" - I can further prune my search space by removing partners that dont end with "ba"
        * And so forth...
    * A trie lets me search for a string by character - independently of the number of other potential strings
    * $O(|V|)$ time for each candidate, where $|V|$ is the length of the candidate
    * Total runtime: $O(cn)$, where $c$ is the length of the longest string

Sources: [1](https://fizzbuzzed.com/top-interview-questions-5/)

In [77]:
from collections import defaultdict
def palindromePairs(words):
    def isPalindrome(s):
        l = 0
        r = len(s)-1
        while l <= r:
            if s[l] != s[r]:
                return False
            l += 1
            r -= 1

        return True
    
            
    class Trie():
        def __init__(self):
            self.paths = defaultdict(Trie)
            self.endIndex = -1
            self.palindromesBelow = []
        
        def addWord(self, word, index):
            trie = self
            for i, w in enumerate(reversed(word)):
                if isPalindrome(word[0:len(word)-i]):
                    trie.palindromesBelow.append(index)
                trie = trie.paths[w]
                
            trie.endIndex = index
        
        def printTrie(self):
            if self.endIndex != -1:
                print(self.endIndex)
            for path in self.paths:
                print(path, end = "")
                self.paths[path].printTrie()
                      
    def buildTrie(words):
        trie = Trie()
        for i, word in enumerate(words):
            trie.addWord(word, i)
        return trie
    
    def getPalindromesForWord(trie, word, index):
        output = []
        while word:
            if trie.endIndex != -1 and isPalindrome(word):
                output += [trie.endIndex]
            if word[0] not in trie.paths:
                return output
            trie = trie.paths[word[0]]
            word = word[1:]
            
        if trie.endIndex >= 0:
            output += [trie.endIndex]
        output += trie.palindromesBelow
        return output
    
    trie = buildTrie(words)
    output = []
    for i, word in enumerate(words):
        candidates = getPalindromesForWord(trie, word, i)
        output += [[i,c] for c in candidates if i != c]
    return output
       
print(palindromePairs(["acbe", "ca", "bca", "bbac"]))    

[[0, 2], [1, 3]]


## [Word search](https://leetcode.com/problems/word-search-ii/)

Given a list of words and a matrix of characters, find all the words that are in the matrix.
### Approach
* Brute force approach
    * For each word, run a DFS from each cell to check if the word exists
    * $O(n(n+m))$ runtime
* What could make it better?
    * Ideally, we want to discover all our words in just one DFS parse of the board
    * A trie is our best bet! Instead of going through n words, we can use the trie to determine at any point during the DFS if we've discovered a word

In [123]:
def findWords(board, words):
    class Trie():
        def __init__(self):
            self.word = None
            self.children = {}
        
        def insert(self, word):
            trie = self
            for w in word:
                trie = trie.children.setdefault(w,Trie())
            trie.word = word
        
        def printTrie(self):
            for child in self.children:
                print(child, end = "")
                self.children[child].printTrie()
    
    def _findWords(board, r, c, t, words_found):
        if t.word:
            words_found += [t.word]
            t.word = None
        # If cell is out of bounds, doesn't match word or has been visited already, return
        if r < 0 or c < 0 or r >= len(board) or c >= len(board[0]) or board[r][c] == "#":
            return
        # If char doesn't match any of the words in the trie, return
        ch = board[r][c]
        if ch not in t.children:
            return
        t = t.children[ch]
        board[r][c] = "#"
        _findWords(board, r+1, c, t, words_found)
        _findWords(board, r-1, c, t, words_found)
        _findWords(board, r, c-1, t, words_found)
        _findWords(board, r, c+1, t, words_found)
        board[r][c] = ch

    t = Trie()
    for word in words:
        t.insert(word)
    
    words_found = []
    for r in range(len(board)):
        for c in range(len(board[0])):
            _findWords(board, r, c, t, words_found)
    return words_found
 
board = [["o","a","a","n"],
         ["e","t","a","e"],
         ["i","h","k","r"],
         ["i","f","l","v"]]
words = ["oath","pea","eat","rain"]
print(findWords(board, words))
board = [["b","b","a","a","b","a"],
         ["b","b","a","b","a","a"],
         ["b","b","b","b","b","b"],
         ["a","a","a","b","a","a"],
         ["a","b","a","a","b","b"]]
words = ["abbbababaa"]
print(findWords(board, words))



['oath', 'eat']
['abbbababaa']


# Dynamic programming
## [Maximal square](https://leetcode.com/problems/maximal-square/)
### Approach
Backtracking:
* Find all possible largest square areas
* If one is a square - return
* Else recurse on all possible smaller squares

In [71]:
def maximalSquare(matrix): 
        def printSquare(square):
            for row in square:
                print(" ".join(row))
            print("")
        def printSquares(squares): 
            for s in squares:
                printSquare(s)
        def allPossibleSquares(square, width):
            squares = []
            for i in range(len(square) - width + 1):
                for j in range(len(square[0]) - width + 1):
                    sq = [[square[a][b] for b in range(j, j + width)] for a in range(i, i + width)]
                    squares.append(sq)
            return squares
                
        def isValidSquare(square):
            for row in square:
                for cell in row:
                    if cell != "1":
                        return False
            return True
        
        def hasBeenChecked(square):
            for row in square:
                for cell in row:
                    if cell != "1":
                        return False
            return True
        
        def _maximalSquare(square, width):
            maxArea = 0
            if width == 0:
                return 0
            for square in allPossibleSquares(matrix, width):
                if isValidSquare(square):
                    return width**2
                else:
                    maxArea = max(maxArea, _maximalSquare(square, width-1))
            return maxArea

            
        
        
        maxArea = 0
        width = min(len(matrix), len(matrix[0]))
        for square in allPossibleSquares(matrix, width):
            if isValidSquare(square):
                return len(square)**2
            else:
                maxArea = max(maxArea, _maximalSquare(square, width-1))
        return maxArea

matrix = [["1","0","1","0","0"],
          ["1","0","1","1","1"],
          ["1","1","1","1","1"],
          ["1","0","0","1","0"]]
maximalSquare(matrix)

4

Note: Work on DP approach. 

## [Regex matching](https://leetcode.com/problems/regular-expression-matching/)
Given an input string (`s`) and a pattern (`p`), implement regular expression matching with support for `'.'` and `'*'`.

`'.'` Matches any single character.  
`'*'` Matches zero or more of the preceding element.

In [209]:
def isMatch(s, p):
    return _isMatch(s,p)


# s: aa p: a* ==> true
    # 
def _isMatch(s, p):
    if not p:
        return not s
    if p[-1] == "*":
        if _isMatch(s,p[:-2]):
            return True
        if s and (s[-1] == p[-2] or p[-2] == '.') and _isMatch(s[:-1], p):
            return True
    if s and (s[-1] == p[-1] or p[-1] == '.') and _isMatch(s[:-1], p[:-1]):
            return True
    else:
        return False
    
print(isMatch("aa", "a")) 
print(isMatch("aa", "a*")) 
print(isMatch("mississippi", "mis*is*p*.")) 

False
True
False


# String
## [Basic calculator](https://leetcode.com/problems/basic-calculator-ii/)
The expression string contains only non-negative integers, +, -, *, / operators and empty spaces . The integer division should truncate toward zero.

In [257]:
import math

def calculate(s):
    s = "".join(s.split())
    ops = set(list("*/+-"))
    st = []
    
    i = 0
    while i < len(s):
        if s[i] in ops:
            if s[i] == "*":
                a = st.pop()
                b = ""
                i += 1
                while i < len(s) and s[i] not in ops:
                    b += s[i]
                    i += 1    
                st.append(a*int(b))
            elif s[i] == "/":
                a = st.pop()
                b = ""
                i += 1
                while i < len(s) and s[i] not in ops:
                    b += s[i]
                    i += 1  
                st.append(math.floor(a/int(b)))
            elif s[i] in "+-":
                st.append(s[i])
                i += 1
        else:
            num = ""
            while i < len(s) and s[i] not in ops:
                num += s[i]
                i += 1
            st.append(int(num))
    
    st = [i for i in reversed(st)]
    while len(st) > 1:
        a = st.pop()
        op = st.pop()
        b = st.pop()
        if op == "+":
            st.append(a+b)
        else:
            st.append(a-b)
    return st.pop()
            
print(calculate("3+2*2"))    
print(calculate("1-1+1"))    
print(calculate("2*3+4")) 
print(calculate("1*2-3/4+5*6-7*8+9/10")) 


    
    

7
1
10
-24


## Text justification
Given an array of words and a width maxWidth, format the text such that each line has exactly maxWidth characters and is fully (left and right) justified.

You should pack your words in a greedy approach; that is, pack as many words as you can in each line. Pad extra spaces ' ' when necessary so that each line has exactly maxWidth characters.

Extra spaces between words should be distributed as evenly as possible. If the number of spaces on a line do not divide evenly between words, the empty slots on the left will be assigned more spaces than the slots on the right.

For the last line of text, it should be left justified and no extra space is inserted between words.

* Approach:
    * If words + 1 space all fit - put em
    * Otherwise 


In [261]:
def fullJustify(words, maxWidth):
    return ""
    

# LinkedLists
## [Merge k sorted lists](https://leetcode.com/problems/merge-k-sorted-lists/)

In [220]:
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None
        
def mergeKLists(lists):
        # At each point - compare all the heads of the linkedlists and pick the min
        node = ListNode(None)
        head = node
        prev = None
        
        lists = [lists[i] for i in range(len(lists)) if lists[i]]
        
        while lists != []:
            # Find minimum head of lists
            minN = float('inf')
            minInd = 0
            for i in range(len(lists)):
                n = lists[i]
                if n.val < minN:
                    minN = n.val
                    minInd = i
            
            if not node or minInd >= len(lists):
                continue
                
            # Add it to final list
            node.val = minN
            node.next = ListNode(None)
            prev = node
            node = node.next
            
            
            # Remove it from initial list
            if lists[minInd].next:
                lists[minInd] = lists[minInd].next
            else:
                del lists[minInd]
        
        if prev:
            prev.next = None
        if head.val is not None:
            return head
        return None
                
l = ListNode(0)
l.next = ListNode(2)
l.next.next = ListNode(5)
        
print(mergeKLists([l]))                   

## [Alien dictionary](https://evanyang.gitbooks.io/leetcode/content/LeetCode/alien_dictionary.html)
There is a new alien language which uses the latin alphabet. However, the order among letters are unknown to you. You receive a list of words from the dictionary, where words are sorted lexicographically by the rules of this new language. Derive the order of letters in this language.

For example, given the following words in dictionary,
```
[
  "wrt",
  "wrf",
  "er",
  "ett",
  "rftt"
]
```
The correct order is: `"wertf"`.

Note:
You may assume all letters are in lowercase.
If the order is invalid, return an empty string.
There may be multiple valid order of letters, return any one of them is fine.

### Approach
* What do we know?
    * First letters of words are in order
    * When first letters are the same, second letters of words are in order
    * And so on
    * In this case, we know w --> e --> r and t --> f and r --> t
* How can I put these in order?
    * Put the first letters of each word in a DAG
    * Maintain "buckets" mapping the first letter --> the words starting with that letter
    * Then, for each bucket, remove first letter and call function on subgroup with the DAG
    * Topological sort to figure out the order
        * Loop through the DAG to find indegree of each vertex
        * Add all the "starting points" (vertices with indegree 0) to a set
        * Topological sort using BFS


In [259]:
from collections import defaultdict


def letterOrder(alienwords):
    dag = defaultdict(list)
    createDAG(alienwords, dag)
    
    deg = {}
    for ch in dag:
        for c in dag[ch]:
            deg[c] = deg.get(c, 0) + 1
    
    starts = set([k for k in dag if k not in deg])
    order = ""
    while starts:
        s = starts.pop()
        order += s
        for nbr in dag[s]:
            deg[nbr] -= 1
            dag[s].remove(nbr)
            if deg[nbr] == 0:
                starts.add(nbr)
    print(order)
    
    

def createDAG(words, g):
    buckets = defaultdict(list)
    order = []
    for w in range(len(words)):
        if not words[w]:
            continue
        first_ch, rest = words[w][0], words[w][1:]
        order.append(first_ch)
        buckets[first_ch].append(rest)
    
    # Add characters to DAG
    for i in range(len(order)-1):
        if order[i] not in g and order[i+1] != order[i]:
            g[order[i]].append(order[i+1])
    
    # Recurse on buckets
    for b in buckets:
        createDAG(buckets[b], g)
    

aw = [
  "wrt",
  "wrf",
  "er",
  "ett",
  "rftt"
]
letterOrder(aw)

wertf
